**Random values for repeatable simulations**

Ronald L. Rivest

October 20, 2020

Used in **SimRisk** simulation of covid-19 disease spread.

In [1]:
module SimRandom

"""Pseudorandom permutation = Key Derivation Function
   (see CLRS "Introduction to Algorithms", Hashing chapter, 4th edition)
"""

export myKDF
export myRand

"""
    myKDF(k, x)

Key Derivation Function.
Return pseudorandom UInt64 value for input x, depending
on (key or seed) parameter k.  Based on RC6 and RC6mix.  myKDF is 
one-to-one (a permutation of x values) for any fixed odd k;
it is like a block cipher. Tight code that runs in registers.
"""
function myKDF(k, x)::UInt64
    r = 4                              # number of rounds
    aec = 0x597148a0b6de3cf2           # arbitrary even constant
    k = unsigned(aec * k + 1)          # k is now odd
    y = unsigned(x)
    for i in 1:r
        y = y * (aec*y + k)            # one-to-one fn of x (as in RC6)
        y = ((y >>> 32) | (y << 32))   # swap 32-bit halves
    end
    return y + 0x2381457096facbed      # eliminates "0 is fixpoint for x" issue
end    

"""
    myRand(k)
    myRand(k, x)

Return float pseudorandom value between 0.0 and 1.0 for 
parameter k and input x.  The input x is 64-bit Int or UInt.
Input x may be omitted, in which case it defaults to a large
pseudorandom constant.

# Examples
```jldoctest
julia> rand(1, 4)
0.4125945601478762
```
"""
function myRand(k, x=0xf3251089badc547e)::Float64
    return myKDF(k, x) / typemax(UInt64)
end

function myRand(k, x, range::UnitRange{Int})::Int
    n = range.stop - range.start + 1
    @assert n > 0
    return min(range.stop, range.start + Int(floor(myRand(k, x)*n)))
end

if false
    println(myRand(1))
    println(myRand(1, 4))
    println(myRand(3, 3))
    println(myRand(10, 7))
    for i in 1:20
        println(myRand(1234, 200+i, 3:7))
    end
end

end # of module SimRandom

Main.KDF